In [1]:
!ollama list

NAME                                                        	ID          	SIZE  	MODIFIED   
nomic-embed-text:latest                                     	0a109f422b47	274 MB	2 days ago	
spow12_Qwen2-7B-ko-Instruct-orpo-ver_2.0_wo_chat_Q8_0:latest	b543bb00f540	8.1 GB	3 days ago	
spow12_Qwen2-7B-ko-Instruct-orpo-ver_2.0_wo_chat-F16:latest 	3713f91a3557	15 GB 	3 days ago	
qwen2:1.5b-instruct-q8_0                                    	908c3f054aac	1.6 GB	3 days ago	
qwen2:0.5b-instruct-q8_0                                    	6b8eef84f0bf	531 MB	3 days ago	
llama3.1:8b-instruct-q8_0                                   	9b90f0f552e7	8.5 GB	4 days ago	
joongi007_Ko-Qwen2-7B-Instruct-GGUF:latest                  	7f2cc98a7ef4	15 GB 	4 days ago	
QuantFactory_ko-gemma-2-9b-it-GGUF:latest                   	8f110e7f7e3c	9.8 GB	4 days ago	


In [4]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv('../dot.env')

True

In [8]:
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]
embedding = OpenAIEmbeddings()

# initialize the bm25 retriever and faiss retriever
# metadata를 직접 삽입, doc_list_1*문서의 길이만큼 반복하여 삽입
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)

bm25_retriever.k = 2

doc_list_2 = [
    "You like apples",
    "You like oranges",
]
# initialize the bm25 retriever and faiss retriever
# metadata를 직접 삽입, doc_list_1*문서의 길이만큼 반복하여 삽입

faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)

faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [9]:
docs = ensemble_retriever.invoke("apples")
docs

[Document(metadata={'source': 1}, page_content='I like apples'),
 Document(metadata={'source': 2}, page_content='You like apples'),
 Document(metadata={'source': 1}, page_content='Apples and oranges are fruits'),
 Document(metadata={'source': 2}, page_content='You like oranges')]

In [10]:
from langchain_core.runnables import ConfigurableField
faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs_faiss",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
docs

[Document(metadata={'source': 1}, page_content='I like apples'),
 Document(metadata={'source': 2}, page_content='You like apples'),
 Document(metadata={'source': 1}, page_content='Apples and oranges are fruits')]